In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/')
sys.path.append('/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF')
sys.path.append('/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/NCF')
import Loader
import MF
from neuralCF import NCF
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class Result:
  def __init__(self, batch_size):
    DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/'
    loader1 = Loader.Loader(DIR, 1, 4) 
    loader2 = Loader.Loader(DIR, 2, 4)
    loader3 = Loader.Loader(DIR, 3, 4)
    loader4 = Loader.Loader(DIR, 4, 4)
    loader5 = Loader.Loader(DIR, 5, 4)
    loader1.load_dataset()
    loader2.load_dataset()
    loader3.load_dataset()
    loader4.load_dataset()
    loader5.load_dataset()
    self.loaders = [loader1, loader2, loader3, loader4, loader5]


    DIR2 = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/raw_data/'
    problemMeta = pd.read_csv(os.path.join(DIR2, "problemMeta.csv"))
    self.probid2level = {row[0]:row[5] for row in problemMeta.values}

    #self.mf = MF.MF(loader1.users_no, loader1.prob_no, loader1.useridx2level, loader1.probidx2level)
    self.ncf = NCF(loader1.users_no, loader1.prob_no, loader1.useridx2level, loader1.probidx2level)
    #BESTMODEL_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF/best_model/cluster'
    BESTMODEL_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/NCF/best_model/cluster'
    model1 = tf.keras.models.load_model(BESTMODEL_DIR+'1')
    model2 = tf.keras.models.load_model(BESTMODEL_DIR+'2')
    model3 = tf.keras.models.load_model(BESTMODEL_DIR+'3')
    model4 = tf.keras.models.load_model(BESTMODEL_DIR+'4')
    model5 = tf.keras.models.load_model(BESTMODEL_DIR+'5')
    self.models = [model1, model2, model3, model4, model5]

    self.batch_size = batch_size


  def get_output(self, id, problemIds):
    cluster = self.get_cluster(problemIds)
    output = self.goto_model(id, problemIds, self.models[cluster-1], cluster)
    return output

  def get_cluster(self, problemIds):
   maxlevel = max([self.probid2level[prob] for prob in problemIds])
   if maxlevel <= 4 and maxlevel >= 1:
     return 1
   elif maxlevel <= 10:
     return 2
   elif maxlevel <= 13:
     return 3
   elif maxlevel <= 16:
     return 4
   else:
     return 5

  def goto_model(self, id, problemIds, model, cluster):
    usridx = self.get_usr_index(id, cluster)
    probidx = self.get_prb_index(problemIds, cluster)
    neg_probidx = self.get_negative_prob(probidx, cluster)
    
    train_usr = np.array([usridx] * len(probidx)).reshape(-1,1)
    train_prb = np.array(probidx).reshape(-1,1)
    train_entry = np.ones_like(train_usr)

    test_usr = np.array([usridx] * len(neg_probidx)).reshape(-1,1)
    test_prb = np.array(neg_probidx).reshape(-1,1)

    weights = model.get_weights()
    for i in range(0, len(train_usr), self.batch_size):
      idxlist = range(i, min(i+self.batch_size, len(train_usr)-1))
      model.fit([train_usr[idxlist], train_prb[idxlist]], train_entry[idxlist], verbose=0)
          
    for i in range(0, len(test_usr), self.batch_size):
      idxlist = range(i, min(i+self.batch_size, len(test_usr)-1))
      pred = model.predict([test_usr[idxlist], test_prb[idxlist]])
      pred = np.concatenate(pred).reshape(-1,1)
    
    filtered = self.ncf.level_filtering(test_usr, test_prb, pred, self.loaders[cluster-1].useridx2level, self.loaders[cluster-1].probidx2level, k=30)
    #filtered = self.mf.level_filtering(test_usr, test_prb, pred, self.loaders[cluster-1].useridx2level, self.loaders[cluster-1].probidx2level, k=30)
    model.set_weights(weights)

    output = self.get_id(filtered[1], cluster)
    return output

  def get_usr_index(self, id, cluster):
    try:
      usridx = self.loaders[cluster-1].userid2idx[id]
    except:
      usridx = self.loaders[cluster-1].users_no + self.loaders[cluster-1].users_free
    return usridx
  
  def get_prb_index(self, problemIds, cluster):
    prbidx = [self.loaders[cluster-1].probid2idx[prob] for prob in problemIds]
    return prbidx
  
  def get_negative_prob(self, problems, cluster):
    return list(set(range(0, self.loaders[cluster-1].prob_no)) - set(problems))

  def get_id(self,problems, cluster):
    prbidx = [self.loaders[cluster-1].probidx2id[prob] for prob in problems]
    return prbidx

In [ ]:
import Result
import warnings
warnings.filterwarnings('ignore')

result = Result(1024)
# id가 없는 경우 기존 아이디랑 겹치지 않게 임의지정해서 실행
result.get_output("gunjung2147", [1152, 2562, 1157, 2438, 2439, 11654, 1546, 11021, 11022, 2577, 9498, 2588, 1316, 1065, 1712, 1330, 2739, 2741, 2742, 10809, 10171, 10172, 10430, 15552, 2753, 4673, 10818, 2884, 10950, 10951, 10952, 8393, 11720, 2941, 1110, 14681, 2908, 10718, 1000, 1001, 3052, 15596, 1008, 2675, 2292, 10869, 5622, 10871, 4344, 10998, 2557, 8958])

In [ ]:
#
result.get_output("gunsong2", [2562, 10757, 1546, 11279, 2577, 2579, 2581, 10773, 11286, 2588, 1065, 2606, 10809, 10816, 2557, 3649, 4673, 10817, 10818, 10828, 1110, 11866, 10845, 2667, 2156, 10866, 2675, 2164, 10869, 10870, 7287, 1655, 10871, 10872, 1149, 1152, 2178, 1157, 9372, 1182, 1697, 1193, 10926, 1712, 2739, 2741, 2742, 2231, 2750, 2751, 10430, 2753, 15552, 10950, 10951, 10952, 8393, 1753, 1260, 10989, 2798, 15596, 2805, 10998, 4344, 8958, 11021, 11022, 2839, 9498, 15649, 15650, 15651, 1316, 15652, 11047, 1330, 9012, 2869, 5430, 2884, 1874, 18258, 4949, 14681, 2908, 4963, 2920, 1912, 2941, 1920, 2438, 1927, 2439, 1929, 9095, 1931, 1932, 11654, 2447, 7569, 17298, 7576, 1966, 1463, 1978, 10171, 10172, 9663, 11720, 11725, 11726, 11729, 5086, 10718, 1000, 1001, 1002, 1003, 3052, 1008, 1010, 1012, 5622, 1021])